In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import autoxgb
from xgboost import plot_importance


ModuleNotFoundError: No module named 'autoxgb'

In [2]:
df = pd.read_csv('../Training_data/data_13_17_1_std.csv')
df = df.drop(['Unnamed: 0','Pe', 'R_total', 'R_calm', 'postime', 'num'], axis = 1)

X = df.drop(['R_added'], axis = 1)
Y = df[['R_added']]

In [3]:
from sklearn.preprocessing import StandardScaler
# 实例化
ss = StandardScaler()
# 特征数据
X = ss.fit_transform(X)
# 目标变量
y = Y
# 先切分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [8]:
from sklearn.tree import DecisionTreeRegressor

tr = DecisionTreeRegressor(max_depth=10)

tr.fit(X_train, y_train)
# 预测值
tr_pre = tr.predict(X_test)
tr_pre =tr_pre.reshape(-1,1)
# 模型评分
print('Score:{:.4f}'.format(tr.score(X_test, y_test)))
# RMSE(标准误差)
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test,tr_pre))))

test_pre = pd.DataFrame({"test" : y_test['R_added'].tolist(),"pre": tr_pre.flatten()})
test_pre

Score:0.2406
RMSE:91.4473


,test,pre
0,378.306892,271.625320
1,176.369357,271.544238
2,286.979469,278.239184
3,131.249981,174.413556
4,209.966490,327.122933
...,...,...
5165,151.468155,239.401062
5166,267.033127,216.749839
5167,371.452145,271.467253
5168,228.390252,215.237580


In [9]:
from sklearn.svm import SVR

linear_svr = SVR(kernel="linear")
linear_svr.fit(X_train, y_train)
linear_svr_pre = linear_svr.predict(X_test)

# 模型评分
print('Score:{:.4f}'.format(linear_svr.score(X_test, y_test)))
# RMSE(标准误差)
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test,linear_svr_pre))))

/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Score:0.0736
RMSE:101.0055


In [10]:
from sklearn import ensemble

gb = ensemble.GradientBoostingRegressor()

gb.fit(X_train, y_train)
gb_pre=gb.predict(X_test)

# 模型评分
print('Score:{:.4f}'.format(gb.score(X_test, y_test)))
# RMSE(标准误差)
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test,gb_pre))))

/home/codespace/.local/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


Score:0.2013
RMSE:93.7869


In [14]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

from xgboost import plot_importance

xgb_model = XGBRegressor(nthread=7)
cv_split = ShuffleSplit(n_splits=6, train_size=0.7, test_size=0.2)
grid_params = dict(
    max_depth = [4, 5, 6, 7],
    learning_rate = np.linspace(0.03, 0.3, 10),
    n_estimators = [100, 200]
)
grid_model = GridSearchCV(xgb_model, grid_params, cv=cv_split, scoring='neg_mean_squared_error')
grid_model.fit(X_train, y_train)

pred = grid_model.predict(X_test)

print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test,pred))))

test_pre = pd.DataFrame({"test" : y_test['R_added'].tolist(),"pre": pred.flatten()})

test_pre

RMSE:67.7264


,test,pre
0,378.306892,261.103302
1,176.369357,256.053802
2,286.979469,249.544113
3,131.249981,159.558075
4,209.966490,261.990417
...,...,...
5165,151.468155,190.843628
5166,267.033127,245.053955
5167,371.452145,283.883514
5168,228.390252,221.937103


### Use AutoXGB

In [11]:
# X_train, X_test, y_train, y_test

df = pd.read_csv('../Training_data/data_13_17_1_std.csv')
df = df.drop(['Unnamed: 0','Pe', 'R_total', 'R_calm', 'postime', 'num'], axis = 1)

msk = np.random.rand(len(df)) < 0.8

train = df[msk]

test = df[~msk]

train.to_csv('train_auxgb.csv')
test.to_csv('test_auxgb.csv')



TypeError: 'Index' object is not callable

In [12]:
from autoxgb import AutoXGB


# required parameters:
train_filename = "train_auxgb.csv"
output = "output"

# optional parameters
test_filename = 'test_auxgb.csv'
task = 'regression'
idx = None
targets = ["R_added"]
features = None
# categorical_features = ['SOG', 'heading', 'draught', 'wind_val', 'wind_direction', ]
categorical_features = train.columns
use_gpu = False
num_folds = 5
seed = 42
num_trials = 100
time_limit = 360
fast = fast

# Now its time to train the model!
axgb = AutoXGB(
    train_filename=train_filename,
    output=output,
    test_filename=test_filename,
    task=task,
    idx=idx,
    targets=targets,
    features=features,
    categorical_features=categorical_features,
    use_gpu=use_gpu,
    num_folds=num_folds,
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast,
)
axgb.train()



ModuleNotFoundError: No module named 'autoxgb'